## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

C:\Users\vu86683\AppData\Local\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def initialise_data(n_rows, random_seed=696):
  """ Generate random dataset for model training """
  # Seed
  np.random.seed(random_seed)
  
  # Generate r and h
  frame = pd.DataFrame({
    'r': np.random.rand(n_rows),
    'h': np.random.rand(n_rows)
  })
  
  # Compute v
  frame['v'] = np.pi*frame['r']**2 * frame['h']
  
  return frame

In [3]:
class tfUtils:
  def __init__(self, num_epochs=50, features=['h','r'], label='v', batch_size=64, queue_capacity=1000):
    self.num_epochs = num_epochs
    self.label = label
    self.batch_size = batch_size
    self.queue_capacity = queue_capacity
    self.features = features
    
  def get_feature_cols(self):
    return [tf.feature_column.numeric_column(k) for k in self.features]

  def get_train_input_fn(self, df):
    return tf.estimator.inputs.pandas_input_fn(
      x = df,
      y = df[self.label],
      batch_size = self.batch_size,
      num_epochs = self.num_epochs,
      shuffle = True,
      queue_capacity = self.queue_capacity
    )

  def get_eval_input_fn(self, df):
    return tf.estimator.inputs.pandas_input_fn(
      x = df,
      y = df[self.label],
      batch_size = self.batch_size,
      shuffle = True,
      queue_capacity = self.queue_capacity
    )

  def get_prediction_input_fn(self, df):
    return tf.estimator.inputs.pandas_input_fn(
      x = df,
      y = None,
      batch_size = self.batch_size,
      shuffle = False,
      queue_capacity = self.queue_capacity
    )
  
  def compute_rmse(self, model, df):
    metrics = model.evaluate(input_fn=self.get_eval_input_fn(df))
    print('RMSE = ', np.sqrt(metrics['average_loss']))

In [4]:
# Generate train and test set
train = initialise_data(20000)
test = initialise_data(500, 969)

In [5]:
# Initialise class for data processing
tf_utils = tfUtils()

In [6]:
# Set logging settings
tf.logging.set_verbosity(tf.logging.WARN)

# Initialise DNN Regressor
model = tf.estimator.DNNRegressor(
          hidden_units=[64, 16, 4],
          feature_columns=tf_utils.get_feature_cols()
          )

# Train model
model.train(input_fn=tf_utils.get_train_input_fn(train));

In [7]:
# Compute error
print(tf_utils.compute_rmse(model, test))

RMSE =  0.0061833714
None
